# Facial Recognition

Predict the labels for image data with a popular dataset
the Labeled Faces in the Wild dataset from the scikit-learn library. The dataset is called
the Olivetti Face dataset and it comprises pictures of famous people's faces, with
appropriate labels

In [6]:
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from time import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

In [7]:
#get faces and resize
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)